In [ ]:
from summer2.functions import time as stf
import pandas as pd
import numpy as np

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
act_trial_reported_rates = {
    2014: 0.0,
    2015: 1.85,
    2016: 1.61,
    2017: 1.51,
    2018: 1.36,
    2018.1: 0.00,
}

In [ ]:
def get_interpolation_rates_from_annual(rates, most_of_year=0.9):
    # Convert keys to float and create initial rates at the start of the period
    start_rates = {float(k): v for k, v in rates.items()}
    
    # Create rates towards the end of the period based on most_of_year
    end_rates = {}
    keys = sorted(rates.keys())  # Sort keys to manage sequence properly
    last_key_index = len(keys) - 1
    
    for i, k in enumerate(keys):
        current_key = float(k)
        current_value = rates[k]
        
        # Determine if we are at the last key or if the next key is a fractional continuation
        if i != last_key_index:
            next_key = keys[i + 1]
            
            # Check if the next key is a fractional continuation of the current year
            if next_key == current_key + 0.1:
                start_rates[next_key] = rates[next_key]
            else:
                # If not, then extend the current rate to the most_of_year point
                end_rate_time = current_key + most_of_year
                if end_rate_time < next_key:
                    end_rates[end_rate_time] = current_value
        else:
            # For the last key, extend the rate to the end of the year and add an extra point
            end_rates[current_key + most_of_year] = current_value
            # This also means maintaining the last rate up to one year later if it's a special non-full-year key
            if current_key != int(current_key):
                end_rates[current_key + 1] = current_value

    # Combine the start and end rates and ensure no duplicates with different values
    interp_rates = {**start_rates, **end_rates}

    # Sort and return the dictionary
    return dict(sorted(interp_rates.items()))


In [ ]:
act_trial_screening_rate = {
        2014.0: 0.0,
        2014.9: 0.0,
        2015.0: 1.85,
        2015.1: 1.61,
        2016.0: 1.61,
        2016.1: 1.51,
        2017.0: 1.51,
        2017.1: 1.36,
        2018.0: 1.36,
        2018.1: 0.00,
    }

In [ ]:
f_sig = stf.get_sigmoidal_interpolation_function(list(act_trial_screening_rate.keys()), list(act_trial_screening_rate.values()), curvature=8)
trial_year = np.linspace(2014,2020.5,10000)
pd.Series(stf.get_time_callable(f_sig)(trial_year), index=trial_year).plot()

In [ ]:
act_control_screening_rate = {
        2017: 0.0,
        2018: 1.9,
        2018.1: 0.0,
}
act_control_screening_rate = get_interpolation_rates_from_annual(act_control_screening_rate)

In [ ]:
f_sig = stf.get_sigmoidal_interpolation_function(list(act_control_screening_rate.keys()), list(act_control_screening_rate.values()), curvature=8)
control_year = np.linspace(2016,2020.5,100000)
pd.Series(stf.get_time_callable(f_sig)(control_year), index=control_year).plot()